In [1]:
import numpy as np
import json

import matplotlib.pyplot as plt
from time import time
import os
import sys

cwd = os.getcwd()

basedir = os.path.abspath(os.path.join(cwd, os.pardir))
if basedir not in sys.path:
    sys.path.insert(0, basedir)


datadir = os.path.join(basedir, 'vessel_data/CSAD')
filepath = os.path.join(datadir, [file for file in os.listdir(datadir) if file.endswith('.json')][1])
print(filepath)

c:\Users\j-ehy\OneDrive - NTNU\NTNU\9 semester\ProsjektOppgave\wavemodel\Wave-Model\vessel_data/CSAD\vessel_json.json


In [2]:
def timeit(func):
    # Function for calculating execution time.
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        end = time()
        print(f"Execution time of '{func.__name__}': {end - start}")
        return result
    return wrapper

In [3]:
# Get the parameters from CSAD

with open(filepath, 'r') as f:
    data = json.load(f)

print(data.keys())

Mrb = np.asarray(data['MRB'])
A = np.asarray(data['A'])
B = np.asarray(data['B'])
C = np.asarray(data['C'])
freqs = np.asarray(data['freqs'])
headings = np.asarray(data['headings'])
Bv = np.asarray(data['Bv'])
# Store RAOs as complex values -> magnitude as real value and phase as imaginary
forceRAO = np.asarray(data['forceRAO']['amp'])[:, :, :, 0] + np.asarray(data['forceRAO']['phase'])[:, :, :, 0]*1j
motionRAO = np.asarray(data['motionRAO']['amp'])[:, :, :, 0] + np.asarray(data['motionRAO']['phase'])[:, :, :, 0]*1j
QTFs = np.asarray(data['driftfrc']['amp'])[:, :, :, 0]
QTFs[0].shape

dict_keys(['main', 'MRB', 'A', 'B', 'C', 'roll', 'freqs', 'headings', 'velocities', 'Bv', 'forceRAO', 'motionRAO', 'driftfrc'])


(47, 36)

In [4]:
from numpy import dtype

# @timeit
# def _full_qtf(self, qtf_headings, qtf_freqs, qtfs, angle, method="Newman"):
#     """
#     Calculate the full QTF matrix for a fixed heading by Newmans approximation or 
#     the geometric mean formulation. 
    
#     #### MAKE THIS RETURN A (36, 47, 47) MATRIX AS - ONE 47 x 47 MATRIX FOR EACH HEADING!!!

#     Note
#     ----
#     Only implemented for unidirectional waves at this moment. 

#     Parameters
#     ----------
#     qtf_headings : 1D-array
#         Headings used in calculation of QTFs
#     qtf_freqs : 1D-array
#         Frequencies used in calculation of QTFs
#     qtf_diag : 1D-array
#         The diagonal terms of the QTF for a given heading.
#     angle : 1D-array
#         Relative wave directional anlge.
#     method : string (default='Newman')
#         Method to be used for approximation.
    
#     Returns
#     -------
#     QTF : 2D-array
#         Full QTF matrix for a fixed heading.

#     References
#     ----------
#     [Newman, 1974] INSERT HER
#     [Standing, Brendling and Wilson]
#     """

#     # First -> pick the values to be used for a given angle.
#     heading_index = np.argmin(np.abs(qtf_headings - angle))
#     qtf_diag = qtfs[:, heading_index, 0]

#     # Second -> interpolate to the frequencies used in wave spectra. (self._freqs)
#     # Chosen to use closes frequency instead - this could be changed to an interpolation scheme instead.
#     freq_indices = [np.argmin(np.abs(qtf_freqs - wave_freq)) for wave_freq in self._freqs]

#     # Third -> generate full QTF matrix based on either Newman approx or Geometric Mean

#     QTF = np.empty((self._N, self._N))
#     for i, freq_i in enumerate(freq_indices):
#         Q_i = qtf_diag[freq_i]
#         for j, freq_j in enumerate(freq_indices):
#             Q_j = qtf_diag[freq_j]

#             if method=="Newman":
#                 QTF[i, j] = 0.5*(Q_i + Q_j)
#             else:
#                 # Using geometric mean approximation
#                 sgn_equal = np.sign(Q_i) == np.sign(Q_j)
#                 if sgn_equal:
#                     QTF[i, j] = np.sign(Q_i)*np.abs(Q_i*Q_j)**(.5)
#                 else:
#                     QTF[i, j] = 0

#     return QTF



@timeit
def full_qtf(qtfs, headings, freqs, wave_freqs, dof=0):
    # Generate full QTF by Newman approximation
    num_headings = len(headings)
    num_freq = len(wave_freqs)
    freq_indices = [np.argmin(np.abs(freqs - w)) for w in wave_freqs]
    Q = np.zeros((num_headings, num_freq, num_freq))
    Qjk = qtfs[dof].copy()
    for i in range(num_headings):
        for j, freq_j in enumerate(freq_indices):
            Q_j = Qjk[freq_j, i]
            for k, freq_k in enumerate(freq_indices):
                Q_k = Qjk[freq_k, i]
                Q[i, j, k] = 0.5*(Q_j + Q_k)
    return Q

@timeit
def full_qtf_alt(qtfs, headings, freqs, wave_freqs, dof=0):
    num_headings = len(headings)
    num_freq = len(wave_freqs)
    freq_indices = [np.argmin(np.abs(freqs - w)) for w in wave_freqs]
    Q = np.zeros((num_headings, num_freq, num_freq))
    Qdiag = qtfs[dof, [freq_indices], :].copy()
    for i in range(len(headings)):
        Q[i] = 0.5*(Qdiag[0, :, i, None] + Qdiag[0, :, i])
    return Q

N = 50 # Number of wave frequencies
wave_freqs = np.linspace(0.1, 3.14, N, endpoint=False)
amps = np.linspace(0, 3, N)
phase = np.random.random(N)*2*np.pi
angles = np.zeros_like(wave_freqs)
# wave_freqs = freqs

print(f"Result diff: {np.sum(full_qtf_alt(QTFs, headings, freqs, wave_freqs, dof=0) - full_qtf(QTFs, headings, freqs, wave_freqs, dof=0))}")

@timeit
def full_qtf_6dof():
    Qd_surge = full_qtf(QTFs, headings, freqs, wave_freqs, dof=0)
    Qd_sway = full_qtf(QTFs, headings, freqs, wave_freqs, dof=1)
    Qd_heave = full_qtf(QTFs, headings, freqs, wave_freqs, dof=2)
    Qd_roll = full_qtf(QTFs, headings, freqs, wave_freqs, dof=3)
    Qd_pitch = full_qtf(QTFs, headings, freqs, wave_freqs, dof=4)
    Qd_yaw = full_qtf(QTFs, headings, freqs, wave_freqs, dof=5)
    return np.vstack([[Qd_surge], [Qd_sway], [Qd_heave], [Qd_roll], [Qd_pitch], [Qd_yaw]])

Qd = full_qtf_6dof()

Execution time of 'full_qtf_alt': 0.0
Execution time of 'full_qtf': 0.10452699661254883
Result diff: 0.0
Execution time of 'full_qtf': 0.07275748252868652
Execution time of 'full_qtf': 0.06411147117614746
Execution time of 'full_qtf': 0.07370281219482422
Execution time of 'full_qtf': 0.05130910873413086
Execution time of 'full_qtf': 0.05476832389831543
Execution time of 'full_qtf': 0.04448199272155762
Execution time of 'full_qtf_6dof': 0.36597371101379395


In [5]:
np.diagonal(Qd[0, 0])

array([4.86661285e-05, 4.86661285e-05, 4.86661285e-05, 1.01960638e-04,
       3.89088820e-04, 7.17648219e-04, 1.26851816e-03, 2.07429047e-03,
       3.64012335e-03, 5.59009633e-03, 7.04724367e-03, 1.18026259e-02,
       1.57227635e-02, 2.14435362e-02, 3.01072378e-02, 4.37648608e-02,
       4.37648608e-02, 6.64250444e-02, 6.64250444e-02, 1.06324207e-01,
       1.06324207e-01, 1.82583235e-01, 1.82583235e-01, 3.45914483e-01,
       3.45914483e-01, 3.45914483e-01, 3.45914483e-01, 7.55869233e-01,
       7.55869233e-01, 7.55869233e-01, 7.55869233e-01, 2.06969239e+00,
       2.06969239e+00, 2.06969239e+00, 2.63168924e+00, 2.63168924e+00,
       4.12212272e+00, 4.12212272e+00, 6.84665024e+00, 6.84665024e+00,
       6.84665024e+00, 1.20650551e+01, 1.20650551e+01, 1.20650551e+01,
       2.28807072e+01, 2.28807072e+01, 2.28807072e+01, 4.64502815e+01,
       4.64502815e+01, 4.64502815e+01])

In [6]:
QTFs[0].shape

(47, 36)

In [7]:
Qd_surge_heading0_diag = QTFs[0, :, 0]

In [8]:
Qd_surge_heading0 = 0.5*(Qd_surge_heading0_diag[:, None] + Qd_surge_heading0_diag)

In [9]:
freqs

array([0.2094395, 0.2513274, 0.3141592, 0.330694 , 0.3490658, 0.3695991,
       0.3926991, 0.4053668, 0.418879 , 0.4333231, 0.4487989, 0.4654211,
       0.4833219, 0.5026549, 0.5235987, 0.5463639, 0.5711986, 0.5983986,
       0.6283185, 0.6613879, 0.6981317, 0.7391983, 0.7853981, 0.837758 ,
       0.8975978, 0.9666439, 1.047197 , 1.142397 , 1.256637 , 1.396263 ,
       1.570796 , 1.795196 , 2.094395 , 2.166616 , 2.302377 , 2.457249 ,
       2.633355 , 2.837934 , 3.07547  , 3.358196 , 3.695992 , 4.109343 ,
       4.630203 , 5.297796 , 6.196435 , 7.453363 , 9.363913 ])

In [10]:
from waves.wave_loads import WaveLoad
from simulator.csad import CSAD_DP_6DOF

vessel = CSAD_DP_6DOF(0.01)

In [11]:


print(vessel._config_file)

waveload = WaveLoad(amps, wave_freqs, phase, angles, vessel._config_file)
waveload._full_qtf_6dof(headings, freqs, QTFs)

c:\Users\j-ehy\OneDrive - NTNU\NTNU\9 semester\ProsjektOppgave\wavemodel\Wave-Model\vessel_data\CSAD\vessel_json.json
50
Execution time of _full_qtf_6dof: 0.0103
50
Execution time of _full_qtf_6dof: 0.0065


array([[[[ 4.86661285e-05,  4.86661285e-05,  4.86661285e-05, ...,
           2.32251651e+01,  2.32251651e+01,  2.32251651e+01],
         [ 4.86661285e-05,  4.86661285e-05,  4.86661285e-05, ...,
           2.32251651e+01,  2.32251651e+01,  2.32251651e+01],
         [ 4.86661285e-05,  4.86661285e-05,  4.86661285e-05, ...,
           2.32251651e+01,  2.32251651e+01,  2.32251651e+01],
         ...,
         [ 2.32251651e+01,  2.32251651e+01,  2.32251651e+01, ...,
           4.64502815e+01,  4.64502815e+01,  4.64502815e+01],
         [ 2.32251651e+01,  2.32251651e+01,  2.32251651e+01, ...,
           4.64502815e+01,  4.64502815e+01,  4.64502815e+01],
         [ 2.32251651e+01,  2.32251651e+01,  2.32251651e+01, ...,
           4.64502815e+01,  4.64502815e+01,  4.64502815e+01]],

        [[ 4.79426604e-05,  4.79426604e-05,  4.79426604e-05, ...,
           2.22254655e+01,  2.22254655e+01,  2.22254655e+01],
         [ 4.79426604e-05,  4.79426604e-05,  4.79426604e-05, ...,
           2.22254655e

In [12]:
# Implempentation in wave loads module corresponds to that done in this notebook
# However - the speed is much faster in the wave module function.
# about 50 times faster actually.
np.sum(full_qtf_6dof() - waveload._full_qtf_6dof(headings, freqs, QTFs))

Execution time of 'full_qtf': 0.07849574089050293
Execution time of 'full_qtf': 0.07392048835754395
Execution time of 'full_qtf': 0.06547904014587402
Execution time of 'full_qtf': 0.06037163734436035
Execution time of 'full_qtf': 0.052338600158691406
Execution time of 'full_qtf': 0.057015180587768555
Execution time of 'full_qtf_6dof': 0.39646482467651367
50
Execution time of _full_qtf_6dof: 0.0069


2.2737367544323206e-12

In [13]:
forceRAO.shape
# Assumption that we have 6 dof x 36 freqs x 36 headings set of RAO transfer functions.

(6, 47, 36)

In [14]:
test_head_indx = np.array([0, 0, 0])
test_freqs_indx = np.array([1, 2, 3])

Q = waveload._full_qtf_6dof(headings, freqs, QTFs)


50
Execution time of _full_qtf_6dof: 0.0107


In [15]:
vessel_specific_waveload = WaveLoad(amps, wave_freqs, phase, np.zeros(len(wave_freqs)), vessel._config_file)

50
Execution time of _full_qtf_6dof: 0.0091


In [16]:
np.sum(Q - vessel_specific_waveload._Q)

0.0

In [17]:
QTFs.shape

(6, 47, 36)

In [18]:
freq_ind = [0, 1, 2]
diag = QTFs[0, [freq_ind], :]
diag[0].shape

(3, 36)

In [19]:
vessel_specific_waveload._Q[:, 0, :, :].shape

(6, 50, 50)

In [20]:
vessel_specific_waveload.second_order_loads(0, 0)

Execution time of second_order_loads: 0.0077


array([ 4.06176884e+03,  6.14105906e-04,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.05303372e-04])

In [21]:
(wave_freqs[:, None] - wave_freqs)

array([[ 0.    , -0.0608, -0.1216, ..., -2.8576, -2.9184, -2.9792],
       [ 0.0608,  0.    , -0.0608, ..., -2.7968, -2.8576, -2.9184],
       [ 0.1216,  0.0608,  0.    , ..., -2.736 , -2.7968, -2.8576],
       ...,
       [ 2.8576,  2.7968,  2.736 , ...,  0.    , -0.0608, -0.1216],
       [ 2.9184,  2.8576,  2.7968, ...,  0.0608,  0.    , -0.0608],
       [ 2.9792,  2.9184,  2.8576, ...,  0.1216,  0.0608,  0.    ]])

In [22]:
forceRAO.shape

(6, 47, 36)

In [23]:
t = 0
x = 0
y = 0

H = np.zeros((6, len(headings), len(wave_freqs), len(wave_freqs)))
Hp = np.zeros_like(H)

In [24]:
heading_indx = np.array([np.argmin(np.abs(headings - a)) for a in angles])
freq_indx = np.array([np.argmin(np.abs(freqs - w)) for w in wave_freqs])


In [25]:
forceRAO[0, [freq_indx], :][0].shape

(50, 36)

In [26]:
vessel_specific_waveload._forceRAOphase.shape

(6, 50, 36)

In [27]:
rel_angle = 0
heading_indx = np.argmin(angles - rel_angle)
vessel_specific_waveload._forceRAOamp[0, :, heading_indx].shape

(50,)

In [28]:
rao_amp = vessel_specific_waveload._forceRAOamp[:, :, heading_indx]
rao_phase = vessel_specific_waveload._forceRAOphase[:, :, heading_indx]
rao_dof_1 = rao_amp[0]*np.cos(wave_freqs*t - phase - rao_phase[0])
rao_dof_2 = rao_amp[1]*np.cos(wave_freqs*t - phase - rao_phase[1])
rao_dof_3 = rao_amp[2]*np.cos(wave_freqs*t - phase - rao_phase[2])
rao_dof_4 = rao_amp[3]*np.cos(wave_freqs*t - phase - rao_phase[3])
rao_dof_5 = rao_amp[4]*np.cos(wave_freqs*t - phase - rao_phase[4])

In [29]:
rao = rao_amp*np.cos(wave_freqs*t - phase - rao_phase)

In [30]:
print(np.sum(rao_dof_1 - rao[0]))
print(np.sum(rao_dof_2 - rao[1]))
print(np.sum(rao_dof_3 - rao[2]))
print(np.sum(rao_dof_4 - rao[3]))
print(np.sum(rao_dof_5 - rao[4]))

0.0
0.0
0.0
0.0
0.0


In [31]:
rao_dof_1 - rao[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
tau_wf = np.zeros(6)
for i in range(6):
    tau_wf[i] = amps@rao[i]
print(tau_wf)

[-5.64792119e+03 -7.81967117e-03  6.63425800e+04  1.60055959e-04
  1.75455571e+04 -1.10373808e-04]


In [33]:
vessel_specific_waveload._Q.shape

(6, 36, 50, 50)

In [34]:
qtest = vessel_specific_waveload._Q[:, 0, :, :]

In [35]:
amp6 = np.vstack([amps, amps, amps, amps, amps, amps])

In [36]:
(qtest@amp6.T).shape

(6, 50, 6)

In [37]:
((vessel_specific_waveload._Q[:, 0, : ,:]*np.exp(1j*vessel_specific_waveload._W))@vessel_specific_waveload._amp)@vessel_specific_waveload._amp

array([ 3.38361557e+04+0.00000000e+00j,  5.11621934e-03+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, -6.85002118e-04+5.42101086e-20j])

In [40]:
(vessel_specific_waveload._amp@(vessel_specific_waveload._Q[:, 0, : ,:]*np.exp(1j*vessel_specific_waveload._W + 1j*vessel_specific_waveload._P))@vessel_specific_waveload._amp)

array([ 2.98860768e+03-4.54747351e-13j,  4.51850506e-04-4.06575815e-20j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, -7.74068034e-05-1.35525272e-20j])

In [41]:
vessel_specific_waveload.second_order_loads(1, 0)

Execution time of second_order_loads: 0.0010


array([ 2.98860768e+03,  4.51850506e-04,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -7.74068034e-05])

In [149]:
tau_sv1 = np.zeros(6)
for i in range(6):
    for j in range(N):
        for k in range(N):
            tau_sv1[i] += np.real(amps[j]*amps[k]*qtest[i, j, k]*np.exp(1j*vessel_specific_waveload._P[j, k]))

tau_sv2 = vessel_specific_waveload.second_order_loads(0, 0)
print(tau_sv1 - tau_sv2)

Execution time of second_order_loads: 0.0010
[1.36424205e-12 1.08420217e-19 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
